Projeto de regressão linear utilizando base de dados dos jogadores do FIFA. Pretendemos encontrar correlação entre o valor do jogador com suas outras características.
Grupo: Felipe Aron, Gabriel Moura e Guilherme Aliperti

In [2]:
import math
import os.path
import pandas as pd
import json
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle
from scipy.stats import linregress
import statsmodels.api as sm

In [3]:
#Transforma os valores em float e sem os caracteres invalidos
df_completo = pd.read_excel("dados_completo.xls")
lista_final_valor = []
lista_final_sal = []

#Para os Valores dos jogadores
for valores in df_completo["Value"]:
    l = valores.split("¬")
    del l[0]
    for x in l:
        carac = x.split("M")
        if len(carac) == 2:
            vf = float(carac[0]) * 1000000
            lista_final_valor.append(vf)
        else:
            carac = x.split("K")
            vf = float(carac[0]) * 1000
            lista_final_valor.append(vf)
            
# Para os salários dos jogadores            
for valores in df_completo["Wage"]:
    l = valores.split("¬")
    del l[0]
    for x in l:
        carac = x.split("M")
        if len(carac) == 2:
            vf = float(carac[0]) * 1000000
            lista_final_sal.append(vf)
        else:
            carac = x.split("K")
            vf = float(carac[0]) * 1000
            lista_final_sal.append(vf)
            
# Cria o Novo Dataframe 
df_valor_idade = pd.DataFrame({"Valor": lista_final_valor, "Overall": df_completo.Overall, "Idade": df_completo.Age,"Salário": lista_final_sal, "Potencial":df_completo.Potential})


In [4]:
# R2 do Overall em relação ao Valor

def lin_regression(x, y):
    m, b, R, p, SEm = linregress(x, y)

    n = len(x)
    SSx = np.var(x, ddof=1) * (n-1)  
    SEb2 = SEm **2 * (SSx/n + np.mean(x)**2)
    SEb = SEb2**0.5

    return m, b, SEm, SEb, R, p

m, b, Sm, Sb, R, p = lin_regression(lista_final_valor, df_completo.Overall)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))



m = 8.171e-07 +- 0.0000
b = 64.3 +- 0.0445

R2 = 0.39188
p of test F : 0.000000


In [5]:
df_valor_idade.describe()

,Idade,Overall,Potencial,Salário,Valor
count,17981.000000,17981.000000,17981.000000,17981.000000,1.798100e+04
mean,25.144541,66.247984,71.190813,11546.966242,2.385390e+06
std,4.614272,6.987965,6.102199,23080.000139,5.353970e+06
min,16.000000,46.000000,46.000000,0.000000,0.000000e+00
25%,21.000000,62.000000,67.000000,2000.000000,3.000000e+05
50%,25.000000,66.000000,71.000000,4000.000000,6.750000e+05
75%,28.000000,71.000000,75.000000,12000.000000,2.100000e+06
max,47.000000,94.000000,94.000000,565000.000000,1.230000e+08


In [39]:
# R2 do Overall em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Overall
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared


0.39187667210821164

Os resultados da regressão linear indicam que o overall do jogador explica quase 40% da variação do seu preço.

Vamos agora avaliar a influência da idade no valor do jogador:

In [40]:
# R2 da Idade em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Idade
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.0062899951432842105

Por fim , vamos analisar a inflûencia do salário de cada jogador no valor dele:

In [48]:
# R2 do Salário em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Salário
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.72336759331866829

In [49]:
# R2 do Potencial em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Potencial
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.34583143211710443

In [50]:
# R2 do Overall,Potencial e Salário juntos em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.loc[:, ("Overall", "Potencial","Salário")]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.75737551312652207

In [53]:
#Lista Atributos dos jogadores
lista_atributos = list(df_completo)
del lista_atributos[0:12]
lista_atributos
del lista_atributos[34:62]

#for atributo in lista_atributos:
#    for valores in df_completo[atributo]:
#        ll = []
#        valores = float(valores)
#        ll.append(valores)
#        df_valor_idade[atributo] : ll

for atributo in lista_atributos:
    Y = df_valor_idade.Valor
    X = df_completo[atributo]
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    results.summary()
    print('Atributo = {:>.4g} ; R-Squared: {:6.4f}'.format(atributo, results.rsquared))   



TypeError: could not convert string to float: '64-5'